# MHPI hydroDL Tutorial

This code contains deep learning code used to model hydrologic systems from soil moisture to streamflow or from projection to forecast.

[![PyPI](https://img.shields.io/badge/pypi-version%200.1-blue)](https://pypi.org/project/hydroDL/0.1.0/)  [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3993880.svg)](https://doi.org/10.5281/zenodo.3993880) [![CodeStyle](https://img.shields.io/badge/code%20style-Black-black)]()


Welcome to our hydroDL tutorial at The Pennsylvania State University! The following notebook is designed to provide a quick start to our project and get you ready to write your own neural networks.

### Obtain hydroDL from Github

In [ ]:
import os
os.chdir("/content/")
import sys
sys.path.append('../')
!rm -rf hydroDLpack # removes any old versions of hydroDL (in case you ran this tutorial previously)
!git clone https://github.com/mhpi/hydroDL.git hydroDLpack
os.chdir("/content/hydroDLpack")
!conda develop .

Cloning into 'hydroDLpack'...
remote: Enumerating objects: 1165, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 1165 (delta 116), reused 143 (delta 85), pack-reused 960
Receiving objects: 100% (1165/1165), 60.71 MiB | 18.50 MiB/s, done.
Resolving deltas: 100% (499/499), done.
Updating files: 100% (265/265), done.
/bin/bash: line 1: conda: command not found


In [ ]:
import os
import torch
import numpy as np

import sys
sys.path.append('..')

#from hydroDL.model.train import trainModel
from hydroDL.model.test import testModel
from hydroDL.post.stat import statError as cal_metric
from hydroDL.data.load_csv import LoadCSV
from hydroDL.utils.norm import re_folder, trans_norm

# set configuration
output_s = "./output/quick_start/"  # output path
csv_path_s = "./example/demo_data/"  # demo data path
all_date_list = ["2015-04-01", "2017-03-31"]  # demo data time period
train_date_list = ["2015-04-01", "2016-03-31"]  # training period
# time series variables list
var_time_series = ["VGRD_10_FORA", "DLWRF_FORA", "UGRD_10_FORA", "DSWRF_FORA", "TMP_2_FORA", "SPFH_2_FORA", "APCP_FORA", ]
# constant variables list
var_constant = ["flag_extraOrd", "Clay", "Bulk", "Sand", "flag_roughness", "flag_landcover", "flag_vegDense", "Silt", "NDVI",
         "flag_albedo", "flag_waterbody", "Capa", ]
# target variable list
target = ["SMAP_AM"]

# generate output folder
re_folder(output_s)

# load your datasets
"""
You can change it with your data. The data structure is as follows:
x_train (forcing data, e.g. precipitation, temperature ...): [pixels, time, features]
c_train (constant data, e.g. soil properties, land cover ...): [pixels, features]
target (e.g. soil moisture, streamflow ...): [pixels, time, 1]

Data type: numpy.float
We have normalized the raw data.
example:
    If the data size is "[pixels, time, features]" or "[pixels, features]", the statistics for 10% to 90% of the data are calculated as follows:

    from hydroDL.utils.norm import cal_statistics
    stat_list = cal_statistics(data, re_extreme=True, percent=10)
    [left_p10, left_p90, mean, std] = stat_list
"""
train_csv = LoadCSV(csv_path_s, train_date_list, all_date_list)
x_train = train_csv.load_time_series(var_time_series)  # data size: [pixels, time, features]
c_train = train_csv.load_constant(var_constant, convert_time_series=False)  # [pixels, features]
y_train = train_csv.load_time_series(target, remove_nan=False)  # [pixels, time, 1]

# validation the result
# load validation datasets
val_date_list = ["2016-04-01", "2017-03-31"]  # validation period
# load your data. same as training data
val_csv = LoadCSV(csv_path_s, val_date_list, all_date_list)
x_val = val_csv.load_time_series(var_time_series)
c_val = val_csv.load_constant(var_constant, convert_time_series=False)
y_val = val_csv.load_time_series(target, remove_nan=False)


In [ ]:
print(type(x_train))
print(x_train.shape, c_train.shape, y_train.shape)
print(x_val.shape, c_val.shape, y_val.shape)

<class 'numpy.ndarray'>
(412, 366, 7) (412, 12) (412, 366, 1)
(412, 365, 7) (412, 12) (412, 365, 1)


In [ ]:
'''
load_data is the Dataloader function for the training function. These functions are typically embedded inside hydroDL package,
but they are pulled out here so that newcomers can understand them without packaging details.
They help to build the minibatch and copy to cuda() device
Here we extract a minibatch from inputs["x", "c", "y"] and wrap the extracted data to data["x", "yTrain"]
'''
from hydroDL.master.master import loadModel
from hydroDL.model.crit import RmseLoss
from hydroDL.model.rnn import CudnnLstmModel
from hydroDL.model.rnn import CpuLstmModel
from hydroDL.model.crit import RmseLoss, NSELossBatch,NSESqrtLossBatch
from hydroDL.model.settings import make_train_settings
import pandas as pd
from time import sleep
from tqdm import trange
import hydroDL.core.logger as logger
from hydroDL.model import crit
import time
log = logger.get_logger("model.train_.train")

def select_subset(
    x, iGrid, iT, rho, c=None, tupleOut=False, LCopt=False, bufftime=0, **kwargs
):
    """
    Selects a subset based on the grid given
    :param x:
    :param iGrid:
    :param iT:
    :param rho:
    :param c:
    :param tupleOut:
    :param LCopt:
    :param bufftime:
    :return:
    """
    nx = x.shape[-1]
    nt = x.shape[1]
    if x.shape[0] == len(iGrid):  # hack
        iGrid = np.arange(0, len(iGrid))  # hack
    if nt <= rho:
        iT.fill(0)
    batchSize = iGrid.shape[0]
    if iT is not None:
        # batchSize = iGrid.shape[0]
        xTensor = torch.zeros([rho + bufftime, batchSize, nx], requires_grad=False)
        for k in range(batchSize):
            temp = x[
                iGrid[k] : iGrid[k] + 1, np.arange(iT[k] - bufftime, iT[k] + rho), :
            ]
            xTensor[:, k : k + 1, :] = torch.from_numpy(np.swapaxes(temp, 1, 0))
    else:
        if LCopt is True:
            # used for local calibration kernel: FDC, SMAP...
            if len(x.shape) == 2:
                # Used for local calibration kernel as FDC
                # x = Ngrid * Ntime
                xTensor = torch.from_numpy(x[iGrid, :]).float()
            elif len(x.shape) == 3:
                # used for LC-SMAP x=Ngrid*Ntime*Nvar
                xTensor = torch.from_numpy(np.swapaxes(x[iGrid, :, :], 1, 2)).float()
        else:
            # Used for rho equal to the whole length of time series
            xTensor = torch.from_numpy(np.swapaxes(x[iGrid, :, :], 1, 0)).float()
            rho = xTensor.shape[0]
    if c is not None:
        nc = c.shape[-1]
        temp = np.repeat(
            np.reshape(c[iGrid, :], [batchSize, 1, nc]), rho + bufftime, axis=1
        )
        cTensor = torch.from_numpy(np.swapaxes(temp, 1, 0)).float()
        if tupleOut:
            if torch.cuda.is_available():
                xTensor = xTensor.cuda()
                cTensor = cTensor.cuda()
            out = (xTensor, cTensor)
        else:
            out = torch.cat((xTensor, cTensor), 2)
    else:
        out = xTensor
    if torch.cuda.is_available() and type(out) is not tuple:
        out = out.cuda()
    return out


def random_index(ngrid, nt, dimSubset, bufftime=0):
    """
    Finds a random place to start inside of the grids
    :param: ngrid: the number of grids
    :param: nt: the number of tiles
    :param: dimSubset: the dimensions of the subset
    :param: bufftime: a buffer
    :returns: the index of the grid and the index of the tile
    """
    batchSize, rho = dimSubset
    iGrid = np.random.randint(0, ngrid, [batchSize])
    iT = np.random.randint(0 + bufftime, nt - rho, [batchSize])
    return iGrid, iT

def save_model(outFolder, model, epoch, modelName="model"):
    modelFile = os.path.join(outFolder, modelName + "_Ep" + str(epoch) + ".pt")
    torch.save(model, modelFile)


def load_model(outFolder, epoch, modelName="model"):
    modelFile = os.path.join(outFolder, modelName + "_Ep" + str(epoch) + ".pt")
    model = torch.load(modelFile)
    return model

def load_data(inputs, settings):
    """

    :param inputs:
    :param settings:
    :return:
    """
    data = {}
    iGrid, iT = random_index(
        settings["ngrid"], settings["nt"], [settings["batchSize"], settings["rho"]], bufftime=settings['bufftime']
    )
    data["x"] = select_subset(inputs["x"], iGrid, iT, settings["rho"], c=inputs["c"], bufftime=settings['bufftime'])
    data["yTrain"] = select_subset(inputs["y"], iGrid, iT, settings["rho"])
    return data, iGrid


def trainModel(
    model,
    x,
    y,
    c,
    lossFun,
    nEpoch=500,
    load_data=load_data,
    miniBatch=[100, 30],
    saveEpoch=100,
    saveFolder=None,
    mode="seq2seq",
    bufftime=0,
):
    optim = torch.optim.Adadelta(model.parameters())
    if saveFolder is not None:
        runFile = os.path.join(saveFolder, "run.csv")
        rf = open(runFile, "w+")
    inputs, settings = make_train_settings(model, x, y, c, nEpoch, miniBatch, bufftime,)
    # wrap data into a dict (inputs).

    if torch.cuda.is_available():
        lossFun = lossFun.cuda()

    # for illustration in this notebook, this wrapping is disabled.
    with trange(1, nEpoch + 1) as pbar:
        for iEpoch in pbar:
            pbar.set_description(f"Training {model.name}")
            lossEp = 0
            t0 = time.time()
            for iIter in range(0, settings["nIterEp"]):
                model.zero_grad()
                dataDict, iGrid = load_data(inputs, settings)
                x = dataDict["x"]
                yp = model(x)

                if type(lossFun) in [NSELossBatch, NSESqrtLossBatch]:
                    loss = lossFun(yp[bufftime:, :, :], dataDict["yTrain"], igrid=iGrid)
                else:
                    loss = lossFun(yp[bufftime:, :, :], dataDict["yTrain"])
                    # Additional handling code if needed

                loss.backward()
                optim.step()
                lossEp = lossEp + loss.item()
                # if iIter % 1 == 0:
                #     print('Iter {} of {}: Loss {:.3f}'.format(iIter, settings["nIterEp"], loss.item()))
            lossEp = lossEp / settings["nIterEp"]
            loss_str = "Epoch {} Loss {:.3f} time {:.2f}".format(
                iEpoch, lossEp, time.time() - t0
            )
            print(loss_str)
            log.debug(loss_str)
            pbar.set_postfix(loss=lossEp)
            sleep(0.1)
            if saveFolder is not None:
                rf.write(loss_str + "\n")
                if iEpoch % saveEpoch == 0:
                    # save model
                    modelFile = os.path.join(
                        saveFolder, "model_Ep" + str(iEpoch) + ".pt"
                    )
                    torch.save(model, modelFile)
    if saveFolder is not None:
        rf.close()

    # return model
    return model #KL: added to test


# define model and loss function
loss_fn = RmseLoss()  # loss function
# select model: GPU or CPU
device = torch.cuda.current_device() if torch.cuda.is_available() else torch.device("cpu" )

# hyperparameter
EPOCH = 100
BATCH_SIZE = 50
RHO = 30
HIDDEN_SIZE = 256
nx = x_train.shape[-1] + c_train.shape[-1]  # update nx, nx = nx + nc
ny = y_train.shape[-1]

if torch.cuda.is_available():
    model = CudnnLstmModel(nx=nx, ny=ny, hiddenSize=HIDDEN_SIZE).to(device)
else:
    model = CpuLstmModel(nx=nx, ny=ny, hiddenSize=HIDDEN_SIZE).to(device)


In [ ]:

# training the model
last_model = trainModel(
    model,
    x_train,
    y_train,
    c_train,
    loss_fn,
    nEpoch=EPOCH,
    miniBatch=[BATCH_SIZE, RHO],
    saveEpoch=100,
    saveFolder=output_s,
    bufftime=0
)
val_epoch = EPOCH # Select the epoch for testing


Training CudnnLstmModel:   1%|          | 1/100 [00:03<05:58,  3.62s/it, loss=0.692]

Epoch 1 Loss 0.692 time 3.51


Training CudnnLstmModel:   2%|▏         | 2/100 [00:07<05:52,  3.59s/it, loss=0.575]

Epoch 2 Loss 0.575 time 3.47


Training CudnnLstmModel:   3%|▎         | 3/100 [00:11<06:10,  3.82s/it, loss=0.534]

...

Epoch 98 Loss 0.310 time 4.40


Training CudnnLstmModel:  99%|█████████▉| 99/100 [06:32<00:03,  3.93s/it, loss=0.309]

Epoch 99 Loss 0.309 time 3.71


Training CudnnLstmModel: 100%|██████████| 100/100 [06:36<00:00,  3.96s/it, loss=0.308]

Epoch 100 Loss 0.308 time 3.71


In [ ]:

# load the model
test_model = loadModel(output_s, epoch=val_epoch)

# set the path to save result
save_csv = os.path.join(output_s, "predict.csv")

# validation
pred_val = testModel(test_model, x_val, c_val, batchSize=len(x_train), filePathLst=[save_csv],)

# select the metrics
metrics_list = ["Bias", "RMSE", "ubRMSE", "Corr"]
pred_val = pred_val.numpy()
# denormalization
pred_val = trans_norm(pred_val, csv_path_s, var_s=target[0], from_raw=False)
y_val = trans_norm(y_val, csv_path_s, var_s=target[0], from_raw=False)
pred_val, y_val = np.squeeze(pred_val), np.squeeze(y_val)
metrics_dict = cal_metric(pred_val, y_val)  # calculate the metrics
metrics = ["Median {}: {:.4f}".format(x, np.nanmedian(metrics_dict[x])) for x in metrics_list]
print("Epoch {}: {}".format(val_epoch, metrics))

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


Epoch 100: ['Median Bias: 0.0046', 'Median RMSE: 0.0316', 'Median ubRMSE: 0.0307', 'Median Corr: 0.8409']
